# Introduction to SQL

### What is SQL
SQL is a standard language used to query/search for data in a SQL standard compliant relational database or RDBMS(Relational Database Management System).

SQL uses a *"declarative"* syntax to easily reason about **what** we'd like to receive but not necessarily **how** to do it. 

### Common SQL Commands

**CRUD Commands:**

- `INSERT` (Create)
- `SELECT` (Read)
- `UPDATE` (Update)
- `DELETE` (Delete)

### Common SQL Datatypes
- int
- bigint
- smallint
- char/nchar(unicode)
- varchar/nvarchar(unicode)
- bit(boolean)
- decimal
- numeric
- float
- datetime
- timestamp
- blob
- binary
- image
- json(newer/proprietary)
- xml(newer/proprietary)

##### Create Databases/Tables
```sql
-- Create a database
CREATE SCHEMA supersecret_production_db;

-- Create a table
CREATE TABLE IF NOT EXISTS pii_data(
    id          int NOT NULL AUTO_INCREMENT PRIMARY KEY,
    ssn         char(11) NOT NULL DISTINCT,
    first_name  varchar(50) NOT NULL,
    last_name   varchar(50) NOT NULL,
    dependents  smallint DEFAULT NULL,
    birthdate   date DEFAULT NULL,
    created_at  timestamp DEFAULT CURRENT_TIMESTAMP
)

-- Create a foriegn key constraint
CREATE TABLE IF NOT EXISTS bank_account(
    id          int NOT NULL AUTO_INCREMENT PRIMARY KEY,
    account     varchar(20) NOT NULL,
    balance     money DEFAULT 0,
    pii_data_id int NOT NULL,
    created_at  timestamp DEFAULT CURRENT_TIMESTAMP
    FOREIGN KEY fk_pii_data(pii_data_id)
    REFERENCES categories(pii_data)
)

```
##### Creating Users
```sql
-- Create a new user with password
CREATE USER 'jaywon'@'localhost' IDENTIFIED BY 'thisisagreatpassword';

-- Create user requiring X509 auth and SHA256 encrypted password and query limits
CREATE USER 'jaywon'@'localhost' IDENTIFIED WITH sha256_password           BY 'thisisevenbetter'
REQUIRE X509 WITH MAX_QUERIES_PER_HOUR 60
ACCOUNT LOCK;
```

##### Assigining permissions to users
```sql
-- Grant all access to all databases (BAD)
GRANT ALL PRIVILEGES ON * . * TO 'jaywon'@'localhost';

-- Grant restricted privileges to local admin user (GOOD)
GRANT SELECT, INSERT, UPDATE, DELETE ON supersecret_production_db.* TO 'jaywon'@'localhost';

-- Grant restricted access to specific tables (EVEN BETTER)
GRANT SELECT ON supersecret_production_db.pii_data TO 'webappuser'@'10.0.0.23';

GRANT SELECT, INSERT, UPDATE, DELETE ON supersecret_production_db.address TO 'webappuser'@'10.0.0.23';

-- Revoke access to a user
REVOKE ALL PRIVILEGES ON supersecret_production_db.top_secret TO 'webappuser'@'10.0.0.23';

-- See user permissions
SHOW GRANTS jaywon;
SHOW GRANTS webappuser;
```

##### Basic Queries
```sql
-- Search for rows in a table
SELECT * 
FROM bank_account 
WHERE balance > 10000;

SELECT first_name, last_name 
FROM pii_data 
WHERE ssn = '444-44-4444';

-- Insert rows into a table
INSERT INTO pii_data(ssn, first_name, last_name)
VALUES('444-44-4444', 'Jay', 'Won');

INSERT INTO pii_data
VALUES('444-44-4444', 'Jay', 'Won', 0, NULL, NULL);

-- Update rows in a table
UPDATE pii_data
SET first_name = 'Jayson'
WHERE id = 12;

-- Delete rows from a table
DELETE FROM bank_account where debt > 100000;
```

##### Joining Tables (JOIN/UNION)
```sql
SELECT pii_data.*, bank_account.* 
FROM pii_data
INNER JOIN bank_account ON pii_data.id = bank_account.pii_data_id
WHERE pii_data.id > 0;
```

##### Destructive Queries
```sql
-- Drop a column on a table
ALTER TABLE money_owed
DROP COLUMN balance;

-- Drop a table
DROP TABLE IF EXISTS money_owed;

-- Drop an entire database
DROP DATABASE supersecret_production_db;
```

### Common Application Development Techniques/Mistakes

#### Techniques
The following are some basic application development concepts in regards to talking to or storing/retrieving data in a database:

- Configuration files
    - Different environments(local, uat, staging, test, production)
- Create a database user for the application to connect as
- Accept user input from forms and other systems to input data into tables in a database to create "dynamic" applications
- Auto-increment integer ID fields for unique identifiers
###### Mapping HTTP request types to CRUD operations

HTTP Method        | SQL Keyword 
:------------- |:-------------
GET      | SELECT
POST      | INSERT
PUT | UPDATE
DEL | DELETE

#### Mistakes
- Plain text password commmitted to source control
- Using the same username/password in different environments
- Not restricting grant permissions of the application user that the application will connect as
- Not sanitizing data to be stored the database
- Creating dynamic query strings and not using ORM or prepared statements, stored procedures, or parameterized queries
- Showing SQL errors back to the user

### Other Data Stores
- "NOSQL" Databases
    - Document Stores
    - Graph Databases
    - Column Databases
    - XML Databases
    - Key/Value Store
    - Blockchains?
    - ....more
- Files
- Proprietary Databases

### Types of SQL Injection
- Blind SQL Injection
- Error based SQL Injection
- Union SQL Injection

**NOTE:** Encoding is often used to get around prevention systems like IDS that doesn't account for all encodings


### Resources
- https://en.wikipedia.org/wiki/SQL
- https://en.wikipedia.org/wiki/NoSQL
- https://www.khanacademy.org/computing/computer-programming/sql
- https://www.w3schools.com/sql/default.asp
- http://www.mysqltutorial.org/
- https://www.owasp.org/index.php/SQL_Injection_Prevention_Cheat_Sheet
- https://www.owasp.org/index.php/Query_Parameterization_Cheat_Sheet
- https://www.owasp.org/index.php/SQL_Injection
- https://www.cybrary.it/0p3n/anatomy-of-error-based-sql-injection/